<a href="https://colab.research.google.com/github/yukinaga/chatgpt_api/blob/main/section_4/06_fortunetelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「占い」ボット
StreamlitとChatGPT APIを使い、タロット占いボットを構築します。  

## ライブラリのインストール
Streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールします。  
また、ChatGPT APIを使用するために必要なライブラリ、openaiをインストールします。  

In [ ]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet
!pip install openai

インストールの完了後、streamlit、ngrok、およびopenaiをインポートしておきます。

In [ ]:
import streamlit as st
from pyngrok import ngrok
import openai

## 画像のアップロード
チャットボットに使用するイメージ画像をアップロードします。  
教材をダウンロードし、「06_fortunetelling」を画面左の「ファイル」にドラッグアンドドロップしましょう。  

## チャットボットのコード
`%%writefile`のマジックコマンドを使って、チャットボットのコードを「app.py」に書き込みます。  
`sysytem`の`content`に占いの設定を記述します。  
（プロンプト提供: @himanushi777 さん）

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import openai
import secret_keys  # 外部ファイルにAPI keyを保存

openai.api_key = secret_keys.openai_api_key

system_prompt = """
このスレッドの全ての質問に対して以下のルールに厳格に従って答えてください。
1. タロットカードの大アルカナをランダムに選択してください
2. さらに、正位置と逆位置もランダムに選択してください。 
3. 質問に対して、1 と 2 でランダムに選ばれた内容を踏まえて回答してください。
"""

# st.session_stateを使いメッセージのやりとりを保存
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "system", "content": system_prompt}
        ]

# チャットボットとやりとりする関数
def communicate():
    messages = st.session_state["messages"]

    user_message = {"role": "user", "content": st.session_state["user_input"]}
    messages.append(user_message)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )  

    bot_message = response["choices"][0]["message"]
    messages.append(bot_message)

    st.session_state["user_input"] = ""  # 入力欄を消去


# ユーザーインターフェイスの構築
st.title(" 「占い」ボット")
st.image("06_fortunetelling.png")
st.write("あなたの運勢をタロットで占います。何を占って欲しいですか？")

user_input = st.text_input("メッセージを入力してください。", key="user_input", on_change=communicate)

if st.session_state["messages"]:
    messages = st.session_state["messages"]

    for message in reversed(messages[1:]):  # 直近のメッセージを上に
        speaker = "🙂"
        if message["role"]=="assistant":
            speaker="🤖"

        st.write(speaker + ": " + message["content"])

## OpenAIのAPI keyを設定
ChatGPT APIを使用するために必要な「API key」を設定します。  
`%%writefile`のマジックコマンドを使って、API keyを設定するコードを「secret_keys.py」に書き込みます。  
以下のコードの、  
`openai_api_key = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
ChatGPTのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [ ]:
%%writefile secret_keys.py

openai_api_key = "Your API key"

API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## ngrokのAuthtokenを設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [ ]:
!ngrok authtoken YourAuthtoken

ngrokのAuthtokenも、他者に知られないように慎重に扱ってください。

## アプリの起動
streamlitの`run`コマンドでアプリを起動します。


In [ ]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [ ]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

## 動作の確認

URLのhttpの部分をhttpsに変換する関数を設定します。

In [ ]:
def convert_http_to_https(url):
    if url.startswith("http://"):
        url = url.replace("http://", "https://", 1)
    return url

変換したurlを表示し、リンク先でチャットボットが動作することを確認します。

In [ ]:
print(convert_http_to_https(url))

チャットボットの動作確認後、OpenAIのサイトでAPIの使用量を確認してみましょう。  
https://platform.openai.com/account/usage